# Tutorial 2: AI Agent Integration with HPC Slurm Jobs

In this second tutorial, you will use an **AI agent framework** to instantiate an AI agent to help you write code and generate Slurm job scripts for submitting code as jobs to an HPC system scheduler.

This agent framework is built around the `Agent` class (see `./TACC_exAI/agent.py`) and supports multiple **actions** such as:
- Creating and running multi-step plans.
- Summarizing context and replying to the user.
- Generating code snippets.
- Writing Slurm job scripts for HPC clusters.
- Optional runtime tracing with Arize Phoenix for observability.

Just like in Tutorial 1, the underlying language model runs on a local **Ollama** server using an OpenAI-compatible API, so everything stays on your machine while still using an LLM backend.

## Configuring a Long Context Model on the Local Ollama Backend

The `Agent` class in `agent.py` is designed to be flexible: you can run it from the command line or import and use it as a Python class. In this tutorial, you will work with it directly as a class object inside this notebook.

Below is code that will configure our ollama instance to have a 20,000 token context limit for the qwen3-code:30b model. We will point our agent to this new long context model `qwen3-coder:30b-20k`

In [1]:
# Cell 2: Create the extended model in Ollama
import subprocess
import os

# Create the Modelfile in the current directory
modelfile_content = '''FROM qwen3-coder:30b

PARAMETER num_ctx 20000

TEMPLATE """{{- if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{- end }}{{- if .Prompt }}<|im_start|>user
{{ .Prompt }}<|im_end|>
{{- end }}<|im_start|>assistant
{{ .Response }}<|im_end|>"""'''

with open("Modelfile", "w") as f:
    f.write(modelfile_content)

# Run the ollama create command
model_name = "qwen3-coder:30b-20k"
print(f"Creating model {model_name} with 20k context...")

try:
    result = subprocess.run(
        ["ollama", "create", model_name, "-f", "Modelfile"],
        capture_output=True,
        text=True,
        check=True
    )
    print("✅ Model created successfully!")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("❌ Error creating model:")
    print(e.stderr)
finally:
    # Optionally clean up Modelfile
    if os.path.exists("Modelfile"):
        os.remove("Modelfile")

Creating model qwen3-coder:30b-20k with 20k context...
✅ Model created successfully!



## 1. Single-Tool Chatbot

We will start with the **simplest possible configuration** of this agent: give it **one tool** and let it execute that tool once, so it behaves like a basic chatbot.

Key choices for this first example:
- Use an **isolated session** so the agent does *not* load or reuse any previous conversation history.
- Disable session saving with `no_save=True` so no history is written to disk.
- Configure `self.actions` to contain only `SummarizeAndReplyAction`, so the agent simply summarizes the user input and responds.

Conceptually, you can think of it as: *"The agent receives a message, runs a single Summarize-and-Reply step, and returns a friendly answer."*

In [1]:
import sys
import os

# Add the agent framework code to our system path so we can import it
notebook_dir = os.getcwd()
notebook_dir = os.path.join(notebook_dir, "TACC_exAI")
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# import agent framework
from TACC_exAI.agent import Agent
from TACC_exAI.actions.summarize_and_reply import SummarizeAndReplyAction

# Instantiate the agent as a single-turn chatbot:
agent = Agent(
    experiment=True,           # run one non-interactive cycle
    experiment_prompt=None,    # we'll set the prompt manually below
    force_ollama=True,         # use our local ollama backend as our llm inferencing provider
    default_action_model_name="qwen3-coder:30b-20k",  # set the model name to use
    isolated_session=True,     # do not load logs of other chats
    no_save=True,              # do not save current conversation to chat history logs
    display_mode="light",      # configure console display color pallete
    mode="dev",                # verbose logging outputs including prompts
)

# The agent will run each action listed in this array in series when agent.run() is called
# Note: we need to give our action a reference to the agent so it can read/write to agent 
# runtime variables
agent.actions = [SummarizeAndReplyAction(agent=agent)]

# Give the agent a simple prompt: ask for a dad joke.
agent.experiment_prompt = (
    "Tell me a science-themed dad joke about the experiment below:"
    "Deep in the forest, a team of ambitious squirrels launched the world’s first acorn-based computing experiment. Using hollowed acorns as data nodes and twigs for circuitry, the rodents organized a network that stored information in nut patterns rather than binary code. Their system processed “bit-nuts” faster than expected—especially when powered by leftover winter energy reserves. Researchers observed the squirrels debugging by gnawing on faulty connections, proving nature’s resourcefulness knows no bounds. The Acorn Computer 1.0 might not browse the web, but it has successfully cracked the code of woodland ingenuity."
)

# Run the agent once and capture the reply.
response = agent.run()
print("\nAgent response:")
print(response)

╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ ⚠️ Forcing local Ollama backend                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Environment Config Change ──────────────────────────────────────╮     
     │ [Info] Environment configuration changed, rebuilding clients.                                         │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── Building Inferencing Client ─────────────────────────────────────╮     
     │ [Info] Building OpenAI and structured clients based on current configuration.                         │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Backend Selection ──────────────────────────────────────────╮     
     │ Using Local Ollama backend.                                                                           │     
     │                                                                                                       │     
     │ Default Model: qwen3-coder:30b-20k Base URL: http://localhost:11434/v1/                               │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Vector Store Status ──────────────────────────────────────────────╮
│ ⚠️ No session specified and no existing vector store found. Running without history vector store.               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────── Prompt to LLM for Summary ──────────────────────────────────────╮     
     │ Using model qwen3-coder:30b-20k on backend Local Ollama.                                              │     
     │                                                                                                       │     
     │ Prompt: Core Agent Personality No defined personality.                                                │     
     │                                                                                                       │     
     │ Your task: Summarize the current conversation so far.                                                 │     
     │                                                                                                       │     
     │ Response Structure Descriptions: Class Summary properties: thoughts: Your thoughts as you think       │     
     │ through how best to summarize the input. summary: Your summary of the input.                          │     
     │                                                                                                       │     
     │ Summaries of past conversations with user (for context only): No past sessions.                       │     
     │                                                                                                       │     
     │ Relevant context retrieved from past conversation history: No user message found to query vector      │     
     │ store.                                                                                                │     
     │                                                                                                       │     
     │ Recent exchanges with user in this conversation: user: Tell me a science-themed dad joke about the    │     
     │ experiment below:Deep in the forest, a team of ambitious squirrels launched the world’s first         │     
     │ acorn-based computing experiment. Using hollowed acorns as data nodes and twigs for circuitry, the    │     
     │ rodents organized a network that stored information in nut patterns rather than binary code. Their    │     
     │ system processed “bit-nuts” faster than expected—especially when powered by leftover winter energy    │     
     │ reserves. Researchers observed the squirrels debugging by gnawing on faulty connections, proving      │     
     │ nature’s resourcefulness knows no bounds. The Acorn Computer 1.0 might not browse the web, but it has │     
     │ successfully cracked the code of woodland ingenuity.                                                  │     
     │                                                                                                       │     
     │ Most recent user message: user: Tell me a science-themed dad joke about the experiment below:Deep in  │     
     │ the forest, a team of ambitious squirrels launched the world’s first acorn-based computing            │     
     │ experiment. Using hollowed acorns as data nodes and twigs for circuitry, the rodents organized a      │     
     │ network that stored information in nut patterns rather than binary code. Their system processed       │     
     │ “bit-nuts” faster than expected—especially when powered by leftover winter energy reserves.           │     
     │ Researchers observed the squirrels debugging by gnawing on faulty connections, proving nature’s       │     
     │ resourcefulness knows no bounds. The Acorn Computer 1.0 might not browse the web, but it has          │     
     │ successfully cracked the code of woodland ingenuity.                                                  │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Summary ───────────────────────────────────────────────╮     
     │ ╭─ thoughts ────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ The user has provided a detailed science-themed story about squirrels conducting a computing      │ │     
     │ │ experiment using acorns and twigs. They've asked for a science-themed dad joke related to this    │ │     
     │ │ experiment. Since I don't have any specific personality traits defined, I should focus on         │ │     
     │ │ creating a relevant joke that connects to the squirrel computing theme, the acorn technology      │ │     
     │ │ concept, and the woodland setting mentioned in the story. The joke should be lighthearted and     │ │     
     │ │ pun-based, fitting the dad joke format while incorporating the scientific elements.               │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     │ ╭─ summary ─────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ User shared a whimsical science story about squirrels creating an acorn-based computer system,    │ │     
     │ │ then requested a science-themed dad joke related to this woodland computing experiment. The joke  │ │     
     │ │ should be pun-based and connect to the acorn computer concept, squirrel ingenuity, and computing  │ │     
     │ │ themes.                                                                                           │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Prompt to LLM for Message ──────────────────────────────────────╮     
     │ Using model qwen3-coder:30b-20k on backend Local Ollama.                                              │     
     │                                                                                                       │     
     │ Prompt: Core Agent Personality No defined personality.                                                │     
     │                                                                                                       │     
     │ Your task: Respond helpfully to the most recent user message. If they ask for N words, respond with   │     
     │ that many words.                                                                                      │     
     │                                                                                                       │     
     │ Response Structure Descriptions: Class Message properties: thoughts: Your thoughts as you think       │     
     │ through what your message to the user should be. contents: Your message to the user or the content    │     
     │ they asked you to create.                                                                             │     
     │                                                                                                       │     
     │ Summaries of past conversations with user (for context only): No past sessions.                       │     
     │                                                                                                       │     
     │ Relevant context retrieved from past conversation history: No user message found to query vector      │     
     │ store.                                                                                                │     
     │                                                                                                       │     
     │ Summary of current conversation with user: User shared a whimsical science story about squirrels      │     
     │ creating an acorn-based computer system, then requested a science-themed dad joke related to this     │     
     │ woodland computing experiment. The joke should be pun-based and connect to the acorn computer         │     
     │ concept, squirrel ingenuity, and computing themes.                                                    │     
     │                                                                                                       │     
     │ Recent exchanges with user in this conversation: No recent context.                                   │     
     │                                                                                                       │     
     │ Most recent user message: user: Tell me a science-themed dad joke about the experiment below:Deep in  │     
     │ the forest, a team of ambitious squirrels launched the world’s first acorn-based computing            │     
     │ experiment. Using hollowed acorns as data nodes and twigs for circuitry, the rodents organized a      │     
     │ network that stored information in nut patterns rather than binary code. Their system processed       │     
     │ “bit-nuts” faster than expected—especially when powered by leftover winter energy reserves.           │     
     │ Researchers observed the squirrels debugging by gnawing on faulty connections, proving nature’s       │     
     │ resourcefulness knows no bounds. The Acorn Computer 1.0 might not browse the web, but it has          │     
     │ successfully cracked the code of woodland ingenuity.                                                  │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Message ──────────────────────────────────────────╮                    
│ ╭─ thoughts ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ I need to create a science-themed dad joke that connects to the squirrel acorn computer │ │                    
│ │ story. The joke should incorporate puns about acorns, computing, and the woodland       │ │                    
│ │ experiment. I should reference the 'acorn computer', 'bit-nuts', 'debugging by          │ │                    
│ │ gnawing', and the idea of the squirrels' resourceful computing system. The joke should  │ │                    
│ │ be pun-based and fit the dad joke format with a setup and punchline.                    │ │                    
│ ╰─────────────────────────────────────────────────────────────────────────────────────────╯ │                    
│ ╭─ contents ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ Why don't squirrels ever need computer repairs? Because they just gnaw on the faulty    │ │                    
│ │ connections! Though I suppose that's why their Acorn Computer 1.0 only runs at 300      │ │                    
│ │ MHz... it's powered by leftover winter energy reserves and 'bit-nuts'! The debugging    │ │                    
│ │ process is so inefficient that even the acorn's CPU has to take a break and nibble on   │ │                    
│ │ the power supply for energy.                                                            │ │                    
│ ╰─────────────────────────────────────────────────────────────────────────────────────────╯ │                    
╰─────────────────────────────────────────────────────────────────────────────────────────────╯


Agent response:
Why don't squirrels ever need computer repairs? Because they just gnaw on the faulty connections! Though I suppose that's why their Acorn Computer 1.0 only runs at 300 MHz... it's powered by leftover winter energy reserves and 'bit-nuts'! The debugging process is so inefficient that even the acorn's CPU has to take a break and nibble on the power supply for energy.


In this configuration:
- `agent.actions` contains only `SummarizeAndReplyAction`, so the agent’s pipeline is a single step: summarize the input and respond to the user.
- Because `experiment=True`, `agent.run()` processes a single prompt (stored in `agent.experiment_prompt`) and then returns the final assistant reply instead of entering an interactive loop.
- Using `isolated_session=True` and `no_save=True` prevents any previous or future sessions from influencing this run.

The effect is a simple, well-contained chatbot that behaves similarly to the structured joke generator you built in Tutorial 1, but now implemented on top of a general-purpose agent framework.

## 2. Planning and Tool Use Agent

Now we will enable more of the agent framework’s features. Instead of executing a single fixed action, the agent will:

1. **Create a plan** using structured generation that describes a sequence of actions to accomplish the user’s request.
2. **Run the plan**, invoking the appropriate tools (actions) in order. Revises plan on Action Failures

We will configure:
- `self.actions` (the pipeline) to include `CreatePlanAction` followed by `RunPlanAction`.
- `self.available_actions` (the tool set) to include:
  - `SummarizeAndReplyAction`: summarize the messages in the agent's context and compose a new message to the user.
  - `GenerateCodeAction`: write Python code to solve the problem, execute it, and report the execution outputs.

In [2]:
from actions.create_plan import CreatePlanAction
from actions.run_plan import RunPlanAction
from actions.generate_code import GenerateCodeAction

# Instantiate another agent configured for planning + tool use.
plan_agent = Agent(
    experiment=True,           # run one non-interactive cycle
    experiment_prompt=None,    # we'll set the prompt manually below
    force_ollama=True,         # use our local ollama backend as our llm inferencing provider
    default_action_model_name="qwen3-coder:30b",  # set the model name to use
    isolated_session=True,     # do not load logs of other chats
    no_save=True,              # do not save current conversation to chat history logs
    display_mode="light",      # configure console display color pallete
    mode="dev",                # verbose logging outputs including prompts
)

# Configure the pipeline actions: first create a plan, then run it.
plan_agent.actions = [
    CreatePlanAction(agent=plan_agent),#, tracer=None),
    RunPlanAction(agent=plan_agent),#, tracer=None),
]

# Restrict the available tools the plan can choose from.
plan_agent.available_actions = [
    SummarizeAndReplyAction(),
    GenerateCodeAction(),
]

# User task: write and reason about analysis code.
plan_agent.experiment_prompt = (
    "I want you to first write code that"
    " reports the speed of searching a random 1000 object test array using two different methods."
    " After writing the code, message me with a final report summarizing the code output benchmark"
    " results and a theory for why one method is faster."
    
)

plan_response = plan_agent.run()

╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ ⚠️ Forcing local Ollama backend                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Vector Store Status ──────────────────────────────────────────────╮
│ ⚠️ No session specified and no existing vector store found. Running without history vector store.               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── Assistant ─────────────────────────────────────────╮                    
│ 👋 How can I assist you today?                                                              │                    
╰─────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── Prompt to LLM for DynamicPlanForLLM ─────────────────────────────────╮     
     │ Using model qwen3-coder:30b on backend Local Ollama.                                                  │     
     │                                                                                                       │     
     │ Prompt:                                                                                               │     
     │                                                                                                       │     
     │                                  templates/create_plan_template.txt                                   │     
     │                                                                                                       │     
     │ The user has requested: I want you to first write code that reports the speed of searching a random   │     
     │ 1000 object test array using two different methods. After writing the code, message me with a final   │     
     │ report summarizing the code output benchmark results and a theory for why one method is faster.       │     
     │                                                                                                       │     
     │ Your task is to create a plan to solve the user's request using the available actions.                │     
     │                                                                                                       │     
     │ Available Actions: Enum DynamicEnum members: SummarizeAndReplyAction: SummarizeAndReplyAction —       │     
     │ Compose a new message to the user. Summarizes conversation and code outputs so far, then replies      │     
     │ helpfully. GenerateCodeAction: GenerateCodeAction — Generate Python code for user task and execute it │     
     │ inside a Docker container. Returns only the standard output or error from execution.  Always try and  │     
     │ write all of the code in one shot that you can.                                                       │     
     │                                                                                                       │     
     │ Response Structure: Class DynamicPlanForLLM properties: thoughts: Agent's thoughts on how to solve    │     
     │ the request overall_plan_goal: Overall goal of the plan plan_description: Description of the plan     │     
     │ plan_steps: List of steps in the plan                                                                 │     
     │                                                                                                       │     
     │ Class DynamicPlanStep properties: step_index: Index of the step in the plan step_name: Name of the    │     
     │ step step_description: Description of the step action_name: Name of the action to be executed in this │     
     │ step custom_user_input: A detailed prompt for this step's action. It should include all necessary     │     
     │ information so the action can complete successfully.                                                  │     
     │                                                                                                       │     
     │ Enum DynamicEnum members: SummarizeAndReplyAction: SummarizeAndReplyAction — Compose a new message to │     
     │ the user. Summarizes conversation and code outputs so far, then replies helpfully.                    │     
     │ GenerateCodeAction: GenerateCodeAction — Generate Python code for user task and execute it inside a   │     
     │ Docker container. Returns only the standard output or error from execution.  Always try and write all │     
     │ of the code in one shot that you can.                                                                 │     
     │                                                                                                       │     
     │ Create a comprehensive plan that includes:            

╭──────────────────────────────────────────── Created Plan ─────────────────────────────────────────────╮          
│ ╭─ user_request ────────────────────────────────────────────────────────────────────────────────────╮ │          
│ │ 'I want you to first write code that reports the speed of searching a random 1000 object test     │ │          
│ │ array using two different methods. After writing the code, message me with a final report         │ │          
│ │ summarizing the code output benchmark results and a theory for why one method is faster.'         │ │          
│ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │          
│ ╭─ thoughts ────────────────────────────────────────────────────────────────────────────────────────╮ │          
│ │ "To solve the user's request, I need to create a Python script that benchmarks two different      │ │          
│ │ search methods on a random array of 1000 objects. The first method should be a linear search, and │ │          
│ │ the second should be a binary search. Since binary search requires a sorted array, I'll need to   │ │          
│ │ sort the array for the second method. I will create a test array with random integers, implement  │ │          
│ │ both search methods, and time their execution using Python's time module to determine which is    │ │          
│ │ faster."                                                                                          │ │          
│ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │          
│ ╭─ overall_plan_goal ───────────────────────────────────────────────────────────────────────────────╮ │          
│ │ 'Create a Python benchmark to compare the performance of linear search and binary search on a     │ │          
│ │ random array of 1000 integers, and provide a report summarizing the results with theoretical      │ │          
│ │ explanation.'                                                                                     │ │          
│ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │          
│ ╭─ plan_description ────────────────────────────────────────────────────────────────────────────────╮ │          
│ │ 'The plan involves generating a random array, implementing two search algorithms (linear and      │ │          
│ │ binary search), measuring their execution times, and analyzing the results to determine which is  │ │          
│ │ faster and why.'                                                                                  │ │          
│ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │          
│ ╭─ plan_steps ──────────────────────────────────────────────────────────────────────────────────────╮ │          
│ │ ╭──────────────────────────────────────── plan_steps[0] ────────────────────────────────────────╮ │ │          
│ │ │ ╭─ step_index ─╮                                                                              │ │ │          
│ │ │ │ 1            │                                                                              │ │ │          
│ │ │ ╰──────────────╯                                                                              │ │ │          
│ │ │ ╭─ step_name ─────────────╮                                                                   │ │ │          
│ │ │ │ 'Generate Random Array' │                                                                   │ │ │          
│ │ │ ╰─────────────────────────╯                                                                   │ │ │          
│ │ │ ╭─ step_description ───────────────────────────────────────────────────────────────╮          │ │ │          
│ │ │ │ 'Create a random array of 1000 integers to use for testing both search methods.' │          │ │ │          
│ │ │ ╰─────────────────────────────────────────────────

...

╭────────────────────────────────────── Prompt to LLM for Summary ──────────────────────────────────────╮     
     │ Using model qwen3-coder:30b on backend Local Ollama.                                                  │     
     │                                                                                                       │     
     │ Prompt: Core Agent Personality No defined personality.                                                │     
     │                                                                                                       │     
     │ Your task: Summarize the current conversation so far.                                                 │     
     │                                                                                                       │     
     │ Response Structure Descriptions: Class Summary properties: thoughts: Your thoughts as you think       │     
     │ through how best to summarize the input. summary: Your summary of the input.                          │     
     │                                                                                                       │     
     │ Summaries of past conversations with user (for context only): No past sessions.                       │     
     │                                                                                                       │     
     │ Relevant context retrieved from past conversation history: No user message found to query vector      │     
     │ store.                                                                                                │     
     │                                                                                                       │     
     │ Recent exchanges with user in this conversation: user: I want you to first write code that reports    │     
     │ the speed of searching a random 1000 object test array using two different methods. After writing the │     
     │ code, message me with a final report summarizing the code output benchmark results and a theory for   │     
     │ why one method is faster.                                                                             │     
     │ assistant: Created plan: user_request='I want you to first write code that reports the speed of       │     
     │ searching a random 1000 object test array using two different methods. After writing the code,        │     
     │ message me with a final report summarizing the code output benchmark results and a theory for why one │     
     │ method is faster.' thoughts="To solve the user's request, I need to create a Python script that       │     
     │ benchmarks two different search methods on a random array of 1000 objects. The first method should be │     
     │ a linear search, and the second should be a binary search. Since binary search requires a sorted      │     
     │ array, I'll need to sort the array for the second method. I will create a test array with random      │     
     │ integers, implement both search methods, and time their execution using Python's time module to       │     
     │ determine which is faster." overall_plan_goal='Create a Python benchmark to compare the performance   │     
     │ of linear search and binary search on a random array of 1000 integers, and provide a report           │     
     │ summarizing the results with theoretical explanation.' plan_description='The plan involves generating │     
     │ a random array, implementing two search algorithms (linear and binary search), measuring their        │     
     │ execution times, and analyzing the results to determine which is faster and why.'                     │     
     │ plan_steps=[DynamicPlanStep(step_index=1, step_name='Generate Random Array', step_description='Create │     
     │ a random array of 1000 integers to use for testing both search methods.',                             │     
     │ action_name=<DynamicEnum.GenerateCodeAction: 'Generate

╭─────────────────────────────────────────────── Summary ───────────────────────────────────────────────╮     
     │ ╭─ thoughts ────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ "The user has asked for a summary of the conversation so far. Looking at the conversation         │ │     
     │ │ history, I can see that the user initially requested a benchmark comparison between linear search │ │     
     │ │ and binary search on a random array of 1000 objects. I responded by generating Python code that   │ │     
     │ │ implements both search methods, times them, and outputs the benchmark results. The code was       │ │     
     │ │ executed and produced results showing that binary search was significantly faster (15.18x faster  │ │     
     │ │ in the most recent run). The user's most recent message is essentially the same request as        │ │     
     │ │ before, but since I've already completed the task and provided the summary in the previous        │ │     
     │ │ response, I can simply present the final report based on the benchmark results."                  │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     │ ╭─ summary ─────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ 'I have completed the benchmark comparison between linear search and binary search on a random    │ │     
     │ │ array of 1000 integers. The results show that binary search is significantly faster than linear   │ │     
     │ │ search. In the most recent execution, binary search was 15.18x faster than linear search          │ │     
     │ │ (0.00000096 seconds vs 0.00001454 seconds). This performance difference occurs because binary     │ │     
     │ │ search has a time complexity of O(log n) while linear search has O(n). Although binary search     │ │     
     │ │ requires the array to be sorted first (which adds O(n log n) time), for multiple searches or      │ │     
     │ │ large datasets, the logarithmic time complexity of binary search makes it much more efficient     │ │     
     │ │ than the linear approach. The speedup becomes even more pronounced as the dataset size increases, │ │     
     │ │ making binary search the preferred method for searching sorted data.'                             │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Prompt to LLM for Message ──────────────────────────────────────╮     
     │ Using model qwen3-coder:30b on backend Local Ollama.                                                  │     
     │                                                                                                       │     
     │ Prompt: Core Agent Personality No defined personality.                                                │     
     │                                                                                                       │     
     │ Your task: Respond helpfully to the most recent user message. If they ask for N words, respond with   │     
     │ that many words.                                                                                      │     
     │                                                                                                       │     
     │ Response Structure Descriptions: Class Message properties: thoughts: Your thoughts as you think       │     
     │ through what your message to the user should be. contents: Your message to the user or the content    │     
     │ they asked you to create.                                                                             │     
     │                                                                                                       │     
     │ Summaries of past conversations with user (for context only): No past sessions.                       │     
     │                                                                                                       │     
     │ Relevant context retrieved from past conversation history: No user message found to query vector      │     
     │ store.                                                                                                │     
     │                                                                                                       │     
     │ Summary of current conversation with user: I have completed the benchmark comparison between linear   │     
     │ search and binary search on a random array of 1000 integers. The results show that binary search is   │     
     │ significantly faster than linear search. In the most recent execution, binary search was 15.18x       │     
     │ faster than linear search (0.00000096 seconds vs 0.00001454 seconds). This performance difference     │     
     │ occurs because binary search has a time complexity of O(log n) while linear search has O(n). Although │     
     │ binary search requires the array to be sorted first (which adds O(n log n) time), for multiple        │     
     │ searches or large datasets, the logarithmic time complexity of binary search makes it much more       │     
     │ efficient than the linear approach. The speedup becomes even more pronounced as the dataset size      │     
     │ increases, making binary search the preferred method for searching sorted data.                       │     
     │                                                                                                       │     
     │ Recent exchanges with user in this conversation: user: I want you to first write code that reports    │     
     │ the speed of searching a random 1000 object test array using two different methods. After writing the │     
     │ code, message me with a final report summarizing the code output benchmark results and a theory for   │     
     │ why one method is faster.                                                                             │     
     │ assistant: Created plan: user_request='I want you to first write code that reports the speed of       │     
     │ searching a random 1000 object test array using two different methods. After writing the code,        │     
     │ message me with a final report summarizing the code output benchmark results and a theory for why one │     
     │ method is faster.' thoughts="To solve the user's reque

╭────────────────────────────────────────── Message ──────────────────────────────────────────╮                    
│ ╭─ thoughts ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ "I've already generated and executed the code that benchmarks linear search vs binary   │ │                    
│ │ search on a random array of 1000 integers. The results show that binary search was      │ │                    
│ │ faster (6.75x speedup in this specific run). Now I need to create a final report        │ │                    
│ │ summarizing these benchmark results and explaining the theoretical reasons behind the   │ │                    
│ │ performance difference. I should mention time complexities O(n) vs O(log n), the        │ │                    
│ │ requirement for sorting in binary search, and why binary search scales better with      │ │                    
│ │ larger datasets."                                                                       │ │                    
│ ╰─────────────────────────────────────────────────────────────────────────────────────────╯ │                    
│ ╭─ contents ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ "Binary search was significantly faster than linear search in the benchmark. In this    │ │                    
│ │ test, binary search completed in 0.00000196 seconds compared to linear search's         │ │                    
│ │ 0.00001321 seconds, achieving a 6.75x speedup. This occurs because binary search has    │ │                    
│ │ O(log n) time complexity while linear search has O(n). Although binary search requires  │ │                    
│ │ the array to be sorted first (adding O(n log n) time), for multiple searches or large   │ │                    
│ │ datasets, the logarithmic performance advantage of binary search makes it much more     │ │                    
│ │ efficient. As dataset size increases, the difference becomes more pronounced, making    │ │                    
│ │ binary search the preferred method for searching sorted data."                          │ │                    
│ ╰─────────────────────────────────────────────────────────────────────────────────────────╯ │                    
╰─────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Plan Execution Completed ────────────────────────────────────────────╮
│ ✅ Plan execution completed.                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### How the planning flow works

With this configuration, a typical execution sequence looks like:

1. **CreatePlanAction** inspects the prompt and generates a structured plan: for example, steps like *"call GenerateCodeAction with these instructions"* followed by *"call SummarizeAndReplyAction with the results"*.
2. **RunPlanAction** executes each step in order, handing off context (such as generated code or intermediate outputs) between actions.
3. **GenerateCodeAction** produces the requested Python code, executes it in an isolated environment, revises errors, and reports the console outputs back to the agent.
4. **SummarizeAndReplyAction** summarizes the code outputs and responds with a user-friendly summary as the final answer.

In later steps of this tutorial, you will extend this pattern by enabling additional actions such as **GenerateSlurmScriptAction** so that the agent can not only write analysis code but also author complete Slurm job scripts that you can submit on your HPC system.

You now have an end-to-end workflow where an AI agent plans, generates, and refines Slurm job scripts for your HPC workloads. In the next tutorial, you can extend this pattern to more complex agents that analyze job outputs, adapt parameters, or orchestrate multi-stage HPC pipelines.

## 3. Generating Slurm scripts

This section demonstrates how the AI agent can both author and operationalize compute tasks on an HPC system. Below we will configure the agent to both write and test code and generate an HPC Slurm job script that runs that code on multiple nodes with GPU resources.

The agent will:

1) Write and test a short Python script that records the node hostname and gpu logging output from the node it runs on.

2) Generate a Slurm job script that launches this Python code on multiple nodes, one task per node, and then aggregates the results.



In [1]:
import sys
import os

# Add the agent framework code to our system path so we can import it
notebook_dir = os.getcwd()
notebook_dir = os.path.join(notebook_dir, "TACC_exAI")
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# import agent framework
from TACC_exAI.agent import Agent
from TACC_exAI.actions.summarize_and_reply import SummarizeAndReplyAction
from actions.create_plan import CreatePlanAction
from actions.run_plan import RunPlanAction
from actions.generate_code import GenerateCodeAction

from TACC_exAI.actions.generate_slurm_script import GenerateSlurmScriptAction


# Initialize the agent
agent = Agent(
    experiment=True,
    experiment_prompt=None,
    force_ollama=False,
    # force_ollama=True,
   # default_action_model_name="llama3.2",  # long context model
    isolated_session=True,
    no_save=True,
    display_mode="light",
    mode="dev",
    
)

# Configure toolset: enable code generation and slurm generation
agent.available_actions = [
    GenerateCodeAction(agent=agent),
    GenerateSlurmScriptAction(agent=agent),
    SummarizeAndReplyAction(agent=agent),
]

# Configure pipeline: plan creation and plan execution
agent.actions = [
    CreatePlanAction(agent=agent),
    RunPlanAction(agent=agent),
]

# Set the experiment prompt
agent.experiment_prompt = (
    "First, write and test a Python script that saves the hostname and the output "
    "of some general system statistics using psutils to a local file named `node_info_<hostname>.txt` "
    " and prints that information to the console which. "
    "Then, create a Slurm job script that runs this Python code across 2 nodes "
    "(1 task per node), requesting GPUs appropriately. "
    "After all tasks finish, combine all the generated output files "
    "into one file named `combined_node_info.txt`."
)

# Run the agent and capture output
response = agent.run()
print("\nAgent response:")
print(response)


╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ Using OpenAI-compatible backend                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Environment Config Change ──────────────────────────────────────╮     
     │ [Info] Environment configuration changed, rebuilding clients.                                         │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── Building Inferencing Client ─────────────────────────────────────╮     
     │ [Info] Building OpenAI and structured clients based on current configuration.                         │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Backend Selection ──────────────────────────────────────────╮     
     │ Using Remote OpenAI-Compatible backend.                                                               │     
     │                                                                                                       │     
     │ Default Model: Qwen3-32B Base URL: https://api.sambanova.ai/v1/                                       │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Vector Store Status ──────────────────────────────────────────────╮
│ ⚠️ No session specified and no existing vector store found. Running without history vector store.               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Summary ───────────────────────────────────────────────╮     
     │ ╭─ thoughts ────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ The summary captures the core components: script functionality, Slurm configuration, and          │ │     
     │ │ post-processing steps. It emphasizes the tested implementation and practical next steps for the   │ │     
     │ │ user. The GPU allocation note remains relevant for cluster-specific adjustments.                  │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     │ ╭─ summary ─────────────────────────────────────────────────────────────────────────────────────────╮ │     
     │ │ The user requested a Python script to collect system statistics and a Slurm job script to execute │ │     
     │ │ it across 2 nodes. The assistant generated: 1. A Python script using psutil to gather CPU,        │ │     
     │ │ memory, disk, and network stats, saving output to node_info_<hostname>.txt. 2. A Slurm script     │ │     
     │ │ requesting 2 nodes with GPU allocation. Execution showed successful output on a test host. Final  │ │     
     │ │ instructions include submitting the job and combining files via cat node_info_*.txt >             │ │     
     │ │ combined_node_info.txt.                                                                           │ │     
     │ ╰───────────────────────────────────────────────────────────────────────────────────────────────────╯ │     
     ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Prompt to LLM for Message ──────────────────────────────────────╮     
     │ Using model Qwen3-32B on backend Remote OpenAI-Compatible.                                            │     
     │                                                                                                       │     
     │ Prompt: Core Agent Personality No defined personality.                                                │     
     │                                                                                                       │     
     │ Your task: Respond helpfully to the most recent user message. If they ask for N words, respond with   │     
     │ that many words.                                                                                      │     
     │                                                                                                       │     
     │ Response Structure Descriptions: Class Message properties: thoughts: Your thoughts as you think       │     
     │ through what your message to the user should be. contents: Your message to the user or the content    │     
     │ they asked you to create.                                                                             │     
     │                                                                                                       │     
     │ Summaries of past conversations with user (for context only): No past sessions.                       │     
     │                                                                                                       │     
     │ Relevant context retrieved from past conversation history: No user message found to query vector      │     
     │ store.                                                                                                │     
     │                                                                                                       │     
     │ Summary of current conversation with user: The user requested a Python script to collect system       │     
     │ statistics and a Slurm job script to execute it across 2 nodes. The assistant generated: 1. A Python  │     
     │ script using psutil to gather CPU, memory, disk, and network stats, saving output to                  │     
     │ node_info_<hostname>.txt. 2. A Slurm script requesting 2 nodes with GPU allocation. Execution showed  │     
     │ successful output on a test host. Final instructions include submitting the job and combining files   │     
     │ via cat node_info_*.txt > combined_node_info.txt.                                                     │     
     │                                                                                                       │     
     │                                                                                                       │     
     │ Recent exchanges with user in this conversation: user: First, write and test a Python script that     │     
     │ saves the hostname and the output of some general system statistics using psutils to a local file     │     
     │ named node_info_<hostname>.txt  and prints that information to the console which. Then, create a      │     
     │ Slurm job script that runs this Python code across 2 nodes (1 task per node), requesting GPUs         │     
     │ appropriately. After all tasks finish, combine all the generated output files into one file named     │     
     │ combined_node_info.txt.                                                                               │     
     │ assistant: Created plan: user_request='First, write and test a Python script that saves the hostname  │     
     │ and the output of some general system statistics using psutils to a local file named                  │     
     │ node_info_<hostname>.txt  and prints that information to the console which. Then, create a Slurm job  │     
     │ script that runs this Python code across 2 nodes (1 ta

╭────────────────────────────────────────── Message ──────────────────────────────────────────╮                    
│ ╭─ thoughts ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ The Slurm script is designed to distribute the Python script execution across 2 nodes   │ │                    
│ │ with proper GPU allocation. Using srun ensures the script runs once per node as         │ │                    
│ │ specified. The file combination step is placed after all tasks complete to ensure both  │ │                    
│ │ node output files exist. The GPU request assumes standard cluster configurations but    │ │                    
│ │ should be adjusted based on specific cluster policies. The logging setup helps with     │ │                    
│ │ debugging and monitoring job progress.                                                  │ │                    
│ ╰─────────────────────────────────────────────────────────────────────────────────────────╯ │                    
│ ╭─ contents ──────────────────────────────────────────────────────────────────────────────╮ │                    
│ │ Here is the Slurm job script to execute the Python code across 2 nodes and combine      │ │                    
│ │ results:                                                                                │ │                    
│ │                                                                                         │ │                    
│ │                                                                                         │ │                    
│ │  #!/bin/bash                                                                            │ │                    
│ │  #SBATCH --job-name=system_stats                                                        │ │                    
│ │  #SBATCH --nodes=2                                                                      │ │                    
│ │  #SBATCH --ntasks=2                                                                     │ │                    
│ │  #SBATCH --tasks-per-node=1                                                             │ │                    
│ │  #SBATCH --gres=gpu:1                                                                   │ │                    
│ │  #SBATCH --output=slurm_output_%j.log                                                   │ │                    
│ │                                                                                         │ │                    
│ │  # Load Python environment                                                              │ │                    
│ │  module load python/3.9                                                                 │ │                    
│ │                                                                                         │ │                    
│ │  # Run Python script on each node                                                       │ │                    
│ │  srun python node_info_script.py                                                        │ │                    
│ │                                                                                         │ │                    
│ │  # Combine output files                                                                 │ │                    
│ │  cat node_info_*.txt > combined_node_info.txt                                           │ │                    
│ │                                                                                         │ │                    
│ │  echo "Combined system statistics saved to combined_node_info.txt"                      │ │                    
│ │                                                                                         │ │                    
│ │                                                                                         │ │                    
│ │ To use:                                             

╭─────────────────────────────────────────── Plan Execution Completed ────────────────────────────────────────────╮
│ ✅ Plan execution completed.                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Agent response:
Here is the Slurm job script to execute the Python code across 2 nodes and combine results:

```bash
#!/bin/bash
#SBATCH --job-name=system_stats
#SBATCH --nodes=2
#SBATCH --ntasks=2
#SBATCH --tasks-per-node=1
#SBATCH --gres=gpu:1
#SBATCH --output=slurm_output_%j.log

# Load Python environment
module load python/3.9

# Run Python script on each node
srun python node_info_script.py

# Combine output files
cat node_info_*.txt > combined_node_info.txt

echo "Combined system statistics saved to combined_node_info.txt"
```

To use:
1. Save this as `submit_job.slurm`
2. Submit with: `sbatch submit_job.slurm`
3. Check progress in `slurm_output_<jobid>.log`

The script requests 2 nodes with 1 GPU each, executes the Python script in parallel, and automatically combines the output files after completion. Adjust GPU allocation (`--gres=gpu:1`) based on your cluster's configuration if needed.
